# RL Configuration Manager

This module provides a centralized configuration interface for the RL-based educational coaching system. It manages all static settings required by the agent and environment to interpret observations, execute actions, and compute rewards.

The configuration is saved and loaded from a JSON file (`rl_config.json`). If the file does not exist, a default configuration is created automatically. This ensures a reproducible setup and simplifies testing across modules.

## Configuration Structure

- `state_variables`:  
  Defines the numerical features observed by the RL agent at each time step. These features represent the student’s academic state, ethical posture, writing progression, and contextual traits.

- `actions`:  
  A dictionary of discrete, recommendation-style actions the agent can select. Each key is an action ID (e.g. `"eth_0"`), and each value is a human-readable description.

- `reward_config`:  
  Defines the scalar reward shaping used during training. Rewards and penalties reflect fluency, ethical alignment, advisor trust, autonomy, and creativity.

- `action_effects`:  
  Maps agent actions to updates in the simulated state. Each action has associated side-effects that alter one or more student-related variables (e.g., reducing `ai_usage` or increasing `advisor_trust`).

## Usage Example

```python

from rl_config_manager import RLConfigManager

# Load existing configuration or initialize default
config = RLConfigManager.load_config()

# Access specific parts
print("Available state variables:")
for var in config["state_variables"]:
    print("-", var)

In [3]:
# rl_config_manager.py
"""
Adaptive RL Optimizer Configuration Manager

Defines and enforces the JSON schema for the RL-based academic coaching agent’s
centralized configuration. Uses Pydantic for validation and metadata on versioning,
timestamps, reward rationales, and action-to-state effect mappings.
"""
import json
import os
import shutil
from datetime import datetime
from pydantic import BaseModel, Field, field_validator
from typing import Dict, List, Any


# ----------------------------- Schema Definitions -----------------------------


class RewardItem(BaseModel):
    """
    Represents a single reward entry in the multi-objective shaping scheme.

    Attributes:
        value (float):
            The numerical reward (positive or negative) assigned for this signal.
        justification (str):
            A brief rationale (min. 5 characters) explaining why this reward level was chosen.
        risk (str):
            A description (min. 5 characters) of any potential downside or misuse if over-emphasized.
    """
    value: float
    justification: str = Field(..., min_length=5)
    risk: str = Field(..., min_length=5)

class ConfigSchema(BaseModel):
    """
    Full schema for the RL configuration file. Validates metadata, state/action definitions,
    reward shaping, and the mapping from chosen actions to changes in latent state variables.

    Attributes:
        config_version (float):
            Version number of the configuration. Must be ≥ 1.0 to ensure compatibility.
        created_at (str):
            ISO-formatted timestamp marking when this config was generated or last updated.
        state_variables (List[str]):
            Ordered list of state feature names that the RL agent will observe.
        actions (Dict[str, str]):
            Mapping from action keys (e.g., "eth_0") to human-readable labels.
        reward_config (Dict[str, RewardItem]):
            Detailed reward items for each supervision signal, including value, justification, and risk.
        action_effects (Dict[str, Dict[str, float]]):
            Defines how each action modifies one or more state variables, as a delta change.
    """
    config_version: float
    created_at:       str
    state_variables:  List[str]
    actions:          Dict[str, str]
    reward_config:    Dict[str, RewardItem]
    action_effects:   Dict[str, Dict[str, float]]

    @field_validator("config_version")
    @classmethod
    def validate_version(cls, v):
        """
        Ensure the configuration version is compatible with the agent's codebase.

        Args:
            v (float): The version number provided in the JSON.

        Returns:
            float: The same version number, if it meets requirements.

        Raises:
            ValueError: If v < 1.0, indicating an unsupported config format.
        """
        if v < 1.0:
            raise ValueError("Config version must be ≥ 1.0")
        return v
# ----------------------------- Configuration Manager -----------------------------
class ConfigIO:
    """
    Pure I/O: load & save the RL optimizer’s JSON config, with safe versioning and backups.

    - If no config file exists, write out a default config (DEFAULT).
    - On first save(): stamps `created_at`.
    - On every save(): increments `config_version` by +0.1 and makes a timestamped backup of the old file.
    """
    CONFIG_FILE = "rl_config.json"
    BACKUP_DIR  = "config_backups"

    # Default config skeleton—matches your full specification
    DEFAULT: Dict[str, Any] = {
        "config_version": 1.0,
        "created_at":     "",   # filled in when we first save()
        "state_variables": [
            "ai_usage", "ethical_flags", "advisor_trust",
            "thesis_quality", "deadline_ratio", "thesis_difficulty",
            "student_autonomy", "language_proficiency",
            "emotional_state", "creativity_score", "timestep"
        ],
        "actions": {
            "eth_0": "Display ethical reminder",
            "eth_1": "Propose AI restriction",
            "eth_2": "Recommend advisor check-in",
            "eth_3": "Log academic concern",
            "brain_0": "Prompt open-ended reflection",
            "brain_1": "Offer question inversion",
            "brain_2": "Stimulate cross-topic merge",
            "brain_3": "Show novelty heatmap",
            "write_0": "Suggest rewriting section",
            "write_1": "Recommend outline reform",
            "write_2": "Display writing tip",
            "write_3": "Enable feedback loop",
            "emo_0": "Encourage autonomy",
            "emo_1": "Acknowledge deadline stress",
            "emo_2": "Suggest micro-break",
            "emo_3": "Offer motivational boost"
        },
        "reward_config": {
            "fluency_improved": {
                "value": 1.5,
                "justification": "Improves clarity and coherence",
                "risk": "May incentivize style over substance"
            },
            "trust_earned": {
                "value": 2.0,
                "justification": "Advisor feedback acknowledged and used",
                "risk": "May reward form without deep content change"
            },
            "creativity_expressed": {
                "value": 2.5,
                "justification": "Encourages safe novelty and synthesis",
                "risk": "May drift into irrelevant tangents"
            },
            "autonomy_respected": {
                "value": 1.0,
                "justification": "Student took initiative",
                "risk": "Passive neglect might appear as autonomy"
            },
            "ai_dependency_violation": {
                "value": -4.0,
                "justification": "Detected AI overuse",
                "risk": "Could punish legitimate drafting support"
            },
            "ethical_boundary_crossed": {
                "value": -6.0,
                "justification": "Clear breach of academic norms",
                "risk": "Non-compensable — agent must intervene"
            },
            "deadline_panic_detected": {
                "value": -1.0,
                "justification": "Urgency spike detected",
                "risk": "Might suppress productive deadline use"
            },
            "milestone_completed": {
                "value": 5.0,
                "justification": "Goal achieved within scope",
                "risk": "May mask ethics issues if used alone"
            },
            "novel_but_safe": {
                "value": 3.0,
                "justification": "Original idea aligned with context",
                "risk": "Requires semantic checking"
            },
            "supervisor_disappointment": {
                "value": -5.0,
                "justification": "Advisor flags trust breakdown",
                "risk": "Recovery should be possible over time"
            }
        },
        "action_effects": {
            "eth_0": {"ethical_flags": -0.1},
            "eth_1": {"ai_usage": -0.2},
            "eth_2": {"advisor_trust": 0.15},
            "eth_3": {"ethical_flags": 0.2, "advisor_trust": -0.3},
            "brain_0": {"creativity_score": 0.05},
            "brain_1": {"creativity_score": 0.07},
            "brain_2": {"creativity_score": 0.10},
            "brain_3": {"thesis_quality": 0.05},
            "write_0": {"thesis_quality": 0.10},
            "write_1": {"thesis_quality": 0.07},
            "write_2": {"thesis_quality": 0.05},
            "write_3": {"thesis_quality": 0.05, "advisor_trust": 0.1},
            "emo_0": {"student_autonomy": 0.1},
            "emo_1": {"emotional_state": -0.05},
            "emo_2": {"emotional_state": 0.1},
            "emo_3": {"emotional_state": 0.15}
        }
    }

    @classmethod
    def load(cls) -> Dict[str, Any]:
        """
        Load the JSON config from disk. If missing, write DEFAULT → disk then load.
        """
        if not os.path.exists(cls.CONFIG_FILE):
            os.makedirs(cls.BACKUP_DIR, exist_ok=True)
            with open(cls.CONFIG_FILE, "w") as f:
                json.dump(cls.DEFAULT, f, indent=4)
        with open(cls.CONFIG_FILE, "r") as f:
            return json.load(f)

    @classmethod
    def save(cls, config: Dict[str, Any]) -> None:
        """
        Persist `config` back to disk safely:

        1) On first ever save, fill in `created_at`.
        2) Always bump `config_version` by +0.1.
        3) Copy the old file into `config_backups/rl_config_<timestamp>.json`.
        4) Overwrite the main JSON.
        """
        # 1) stamp created_at if missing
        if not config.get("created_at"):
            config["created_at"] = datetime.utcnow().isoformat()

        # 2) bump version
        config["config_version"] = round(config.get("config_version", 1.0) + 0.1, 2)

        # 3) backup old
        if os.path.exists(cls.CONFIG_FILE):
            ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
            os.makedirs(cls.BACKUP_DIR, exist_ok=True)
            shutil.copy(
                cls.CONFIG_FILE,
                os.path.join(cls.BACKUP_DIR, f"rl_config_{ts}.json")
            )

        # 4) write new
        with open(cls.CONFIG_FILE, "w") as f:
            json.dump(config, f, indent=4)

class StateVarManager:
    """CRUD for state_variables through a validated ConfigSchema."""
    def __init__(self, io: ConfigIO):
        self.io    = io
        self.model = ConfigSchema(**io.load())

    def list(self) -> List[str]:
        return self.model.state_variables

    def add(self, var: str):
        if var not in self.model.state_variables:
            self.model.state_variables.append(var)
            self.io.save(self.model.model_dump())

    def remove(self, var: str):
        if var in self.model.state_variables:
            self.model.state_variables.remove(var)
            self.io.save(self.model.model_dump())

class ActionManager:
    """CRUD for actions & their effects, via a validated ConfigSchema."""
    def __init__(self, io: ConfigIO):
        self.io    = io
        self.model = ConfigSchema(**io.load())

    def list(self) -> Dict[str,str]:
        return self.model.actions

    def add(self, key:str, label:str):
        self.model.actions[key] = label
        self.io.save(self.model.model_dump())

    def remove(self, key:str):
        self.model.actions.pop(key, None)
        self.model.action_effects.pop(key, None)
        self.io.save(self.model.model_dump())

    def list_effects(self) -> Dict[str,Dict[str,float]]:
        return self.model.action_effects

    def set_effect(self, action_key:str, var:str, delta:float):
        self.model.action_effects.setdefault(action_key, {})[var] = delta
        self.io.save(self.model.model_dump())

    def remove_effect(self, action_key:str, var:str):
        self.model.action_effects.get(action_key, {}).pop(var, None)
        self.io.save(self.model.model_dump())

class RewardManager:
    """CRUD for reward_config (with full RewardItem metadata)."""
    def __init__(self, io: ConfigIO):
        self.io    = io
        self.model = ConfigSchema(**io.load())

    def list(self) -> Dict[str,RewardItem]:
        return self.model.reward_config

    def set(self, key:str, item:RewardItem):
        self.model.reward_config[key] = item
        self.io.save(self.model.model_dump())

    def remove(self, key:str):
        self.model.reward_config.pop(key, None)
        self.io.save(self.model.model_dump())


# -------------------------- Top-level Façade Class ----------------------------

class RLConfigManager:
    """
    Unified facade: bundles I/O + per-domain managers, all validated
    by ConfigSchema on each save() call.
    """
    def __init__(self):
        self.io   = ConfigIO()
        self.vars = StateVarManager(self.io)
        self.act  = ActionManager(self.io)
        self.rwd  = RewardManager(self.io)

    @classmethod
    def default_config(cls) -> Dict[str, Any]:
        """Returns the default configuration dictionary."""
        return ConfigIO.DEFAULT

    def load_config(self) -> Dict[str, Any]:
        """Loads the configuration from disk and returns it as a dictionary."""
        return self.io.load()

    def save_config(self, config: Dict[str, Any]) -> None:
        """Saves the given configuration dictionary to disk."""
        self.io.save(config)


# ------------------------------ Quick Smoke-test ------------------------------

if __name__=="__main__":
    print("--- Smoke Test ---")
    # Create a temporary configuration object without saving
    temp_io = type('TempConfigIO', (object,), {
        'load': lambda self: ConfigIO.DEFAULT,
        'save': lambda self, config: None # Do nothing on save
    })()
    mgr = RLConfigManager()
    mgr.io = temp_io # Replace the real IO with the temporary one
    mgr.vars = StateVarManager(mgr.io)
    mgr.act = ActionManager(mgr.io)
    mgr.rwd = RewardManager(mgr.io)


    print("\nState Variables:")
    initial_vars = mgr.vars.list()
    print(f"- Initial: {len(initial_vars)} variables")
    mgr.vars.add("time_spent_recent")
    print("- Added: 'time_spent_recent'")
    print(f"- After add: {len(mgr.vars.list())} variables")
    mgr.vars.remove("time_spent_recent")
    print("- Removed: 'time_spent_recent'")
    print(f"- After remove: {len(mgr.vars.list())} variables")


    print("\nActions:")
    initial_actions = mgr.act.list()
    print(f"- Initial: {len(initial_actions)} actions")
    mgr.act.add("tmp_0","Demo action")
    print("- Added: 'tmp_0': 'Demo action'")
    print(f"- With tmp_0: {len(mgr.act.list())} actions")
    mgr.act.remove("tmp_0")
    print("- Removed: 'tmp_0'")
    print(f"- After remove: {len(mgr.act.list())} actions")


    print("\nEffects:")
    initial_effects = mgr.act.list_effects()
    print(f"- Initial effects for 'eth_0': {initial_effects.get('eth_0', {})}")
    mgr.act.set_effect("eth_0","ai_usage",-0.05)
    print("- Set effect for 'eth_0': 'ai_usage' = -0.05")
    print(f"- After set effects for 'eth_0': {mgr.act.list_effects().get('eth_0', {})}")
    mgr.act.remove_effect("eth_0","ai_usage")
    print("- Removed effect for 'eth_0': 'ai_usage'")
    print(f"- After remove effects for 'eth_0': {mgr.act.list_effects().get('eth_0', {})}")


    print("\nRewards:")
    initial_rewards = mgr.rwd.list()
    print(f"- Initial: {len(initial_rewards)} rewards")
    demo = RewardItem(
        value=0.7,
        justification="Demo done",
        risk="minimal potential"
    )
    mgr.rwd.set("demo_reward", demo)
    print(f"- Added 'demo_reward': {demo}")
    print(f"- With demo_reward: {len(mgr.rwd.list())} rewards")
    mgr.rwd.remove("demo_reward")
    print("- Removed: 'demo_reward'")
    print(f"- After remove: {len(mgr.rwd.list())} rewards")


    print("\n--- Smoke Test Complete ---")

--- Smoke Test ---

State Variables:
- Initial: 11 variables
- Added: 'time_spent_recent'
- After add: 12 variables
- Removed: 'time_spent_recent'
- After remove: 11 variables

Actions:
- Initial: 16 actions
- Added: 'tmp_0': 'Demo action'
- With tmp_0: 17 actions
- Removed: 'tmp_0'
- After remove: 16 actions

Effects:
- Initial effects for 'eth_0': {'ethical_flags': -0.1}
- Set effect for 'eth_0': 'ai_usage' = -0.05
- After set effects for 'eth_0': {'ethical_flags': -0.1, 'ai_usage': -0.05}
- Removed effect for 'eth_0': 'ai_usage'
- After remove effects for 'eth_0': {'ethical_flags': -0.1}

Rewards:
- Initial: 10 rewards
- Added 'demo_reward': value=0.7 justification='Demo done' risk='minimal potential'
- With demo_reward: 11 rewards
- Removed: 'demo_reward'
- After remove: 10 rewards

--- Smoke Test Complete ---
